Antes de mais nada os notebooks aqui mostrado tiveram como base/foram retirados dos seguintes repositórios: 
 > https://github.com/fchollet/deep-learning-with-python-notebooks 
 
 
 > https://github.com/cdfmlr/Deep-Learning-with-Python-Notebooks
 
 Sugiro fortemente que consultem os códigos originais e em caso de dúvida podem me contatar para conversarmos. 

# Aprendizado profundo com Python

## 7.3 Obtendo o máximo de seus modelos

> Maximize o desempenho do modelo

Se você quer apenas um bom modelo, pode tentar várias arquiteturas de rede sem pensar. Mas se você deseja desenvolver um modelo com excelente desempenho e o que há de mais moderno, precisa considerar as técnicas fornecidas posteriormente.

### Modo de Arquitetura Avançada

Se você deseja construir um modelo excelente, deve conhecer os "padrões de projeto" comumente usados ​​de "conexão residual", "padronização" e "convolução profundamente separável".

#### Conexão residual

> Nota: este parágrafo foi escrito em 7.1, aqui está apenas uma cópia para tornar esta parte mais completa.

A conexão residual (conexão residual) é um componente muito comumente usado agora, ele resolve o problema de desaparecimento e gargalo do gradiente do modelo de aprendizagem profunda em larga escala. Em geral, adicionar conexões residuais a qualquer modelo com mais de 10 camadas pode ajudar.

- Desaparecimento de radiação: há mais camadas passadas e a representação aprendida anteriormente torna-se borrada ou mesmo completamente perdida, fazendo com que a rede falhe no treinamento.
-Indica um gargalo: empilhar camadas, a última camada só pode acessar as coisas aprendidas na camada anterior. Se uma determinada camada for muito pequena (menos informações podem ser inseridas na ativação), as informações serão cardadas e um gargalo aparecerá.

A conexão residual é permitir a saída de uma camada anterior como a entrada de uma camada posterior (criando um atalho na rede). A saída da camada anterior não está conectada com a ativação da última camada, mas é adicionada à ativação da última camada (se a forma for diferente, use uma transformação linear para alterar a ativação da camada anterior na forma de destino).

> Observação: a transformação linear pode usar a camada Densa sem ativação ou usar a convolução 1 × 1 sem ativação no CNN.

`` `python
de camadas de importação keras

x = ...

y = camadas.Conv2D (128, 3, ativação = 'relu', preenchimento = 'mesmo') (x)
y = camadas.Conv2D (128, 3, ativação = 'relu', preenchimento = 'mesmo') (y)
y = camadas.MaxPooling2D (2, passos = 2) (y)

# Formas diferentes, faça transformação linear:
residual = camadas.Conv2D (128, 1, passos = 2, preenchimento = 'mesmo') (x) # Use a convolução 1 × 1 para reduzir linearmente x para ter a mesma forma que y

y = layers.add ([y, residual])
`` `

#### Estandardização

A normalização é usada para tornar as diferentes amostras vistas pelo modelo mais semelhantes entre si, o que ajuda a otimizar e generalizar o modelo.

A padronização de dados mais comum é fazer com que os dados tenham média 0 e variância 1:

`` `python
normalized_data = (data-np.mean (data, axis = ...)) / np.std (data, axis = ...)
`` `

Todos nós sabemos padronizar os dados antes de inseri-los no modelo. Mas depois de cada transformação da rede, devemos também considerar a padronização de dados. Após cada camada de saída, a padronização anterior pode ser destruída, precisamos considerar essa questão.

** Normalização em lote ** (normalização em lote) é uma forma de resolver este problema: durante o treinamento, ele salvará internamente a média móvel exponencial da média e variância de cada lote de dados que foi lido. Portanto, durante o processo de treinamento, a média e a variância mudam com o tempo, e a padronização do lote também pode padronizar os dados de forma adaptativa. Este método ajuda a propagação de gradiente e permite redes mais profundas (semelhantes às conexões residuais).

A normalização em lote no Keras é implementada com a camada `BatchNormalization`, que geralmente é usada após a camada convolucional ou a camada densamente conectada:

`` `python
# Conv
conv_model.add (camadas.Conv2D (32, 3, ativação = 'relu'))
conv_model.add (layers.BatchNormalization ())

# Denso
dense_model.add (layers.Dense (32, ativação = 'relu'))
dense_model.add (layers.BatchNormalization ())
`` `

A camada BatchNormalization recebe um parâmetro de eixo, que especifica qual eixo de recurso deve ser padronizado, e o valor padrão é -1. Para Keras padrão Dense, Conv1D, RNN e Conv2D isso está correto. Mas para Conv2D com `data_format` definido como` "channels_first" `, o eixo do recurso é 1, então você precisa definir` axis = 1`.

#### Convolução separável em profundidade

A camada de convolução separável em profundidade (convolução separável em profundidade), chamada `SeparableConv2D` em Keras, tem a mesma função que Conv2D comum. Mas SeparableConv2D é mais leve, rápido e mais preciso do que Conv2D.

A camada SeparableConv2D realiza a convolução espacial em cada canal de entrada separadamente e, em seguida, integra os resultados de saída por meio da convolução ponto a ponto (convolução 1 × 1). Isso separa o aprendizado de recurso espacial do aprendizado de recurso de canal e, muitas vezes, pode aprender representações melhores usando menos dados. (Isso é muito semelhante ao modelo Xception mencionado acima. Na verdade, a convolução separável profunda é a base da arquitetura Xception)


Para dados com relativamente poucos dados, isso é útil para pinturas treinadas do zero. Por exemplo, o seguinte é uma tarefa de classificação de imagens:

In [1]:
Antes de mais nada os notebooks aqui mostrado tiveram como base/foram retirados dos seguintes repositórios: 
 > https://github.com/fchollet/deep-learning-with-python-notebooks 
 
 
 > https://github.com/cdfmlr/Deep-Learning-with-Python-Notebooks
 
 Sugiro fortemente que consultem os códigos originais e em caso de dúvida podem me contatar para conversarmos. from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers

height = 64
width = 64
channels = 3
num_classes = 10

model = Sequential()
model.add(layers.SeparableConv2D(32, 3, 
                                 activation='relu', 
                                 input_shape=(height, width, channels,)))
model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.GlobalAveragePooling2D())

model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

Para tarefas maiores, o Xception é ainda melhor.

### Otimização de hiperparâmetros

Além de usar padrões de arquitetura avançados, a otimização de hiperparâmetros também merece destaque. Quando escrevemos um modelo, precisamos determinar muitos hiperparâmetros:

Quantas camadas estão empilhadas no modelo, quantas unidades cada camada contém e qual função de ativação é usada ...

Não há regras fixas para ajustar esses hiperparâmetros. É baseado principalmente na intuição e na tentativa e erro. Essa experimentação repetida tem muitas possibilidades e a combinação de vários hiperparâmetros é muito complicada. Ajustar hiperparâmetros o dia inteiro não é uma tarefa humana, é melhor que a máquina faça esse tipo de coisa. Portanto, temos que especificar um processo de ajuste de hiperparâmetro programável:

1. Escolha um conjunto de hiperparâmetros
2. Construa o modelo correspondente
3. Ajuste o modelo em pequenos dados de lote e verifique o desempenho
4. Escolha o próximo conjunto de hiperparâmetros para tentar
4. Repita o processo acima (2 ～ 4)
5. Selecione os melhores hiperparâmetros no experimento acima

A chave para esse processo é que, dados vários conjuntos de hiperparâmetros opcionais, o "desempenho de verificação histórica" ​​é usado para selecionar automaticamente o próximo conjunto de hiperparâmetros a ser avaliado. Você pode usar "pesquisa aleatória simples", "otimização bayesiana", "algoritmo genético" e outros algoritmos para concluir esta tarefa.

Na verdade, é difícil atualizar os hiperparâmetros. O custo computacional de cada tentativa é muito alto, e cada vez você tem que treiná-lo desde o início; e os hiperparâmetros são descontínuos e não diferenciáveis, e algoritmos como gradiente descendente não podem ser usados.

Portanto, na verdade, a otimização automática de hiperparâmetros ainda não está madura e as ferramentas disponíveis são muito limitadas. No entanto, ainda existem muitas bibliotecas que podem ser usadas para ajustar automaticamente os hiperparâmetros de Keras:

- [Hyperas] (https://github.com/maxpumperla/hyperas), esta biblioteca é relativamente antiga e mencionada no livro, mas desde agosto de 2020, ela não foi enviada por vários meses;
- [Keras Tuner] (https://github.com/keras-team/keras-tuner), esta biblioteca é relativamente nova. Eu vi há alguns dias. O número de estrelas é igual a Hyperas.
-...

** Nota **: O ajuste automático do hiperparâmetro consiste basicamente em treinar hiperparâmetros nos dados de verificação. Ao fazer a otimização automática em grande escala de hiperparâmetros, devemos prestar atenção ao problema de sobreajuste do conjunto de verificação!

### Integração do modelo

A combinação de modelos também é uma tecnologia muito poderosa. A integração do modelo refere-se a reunir os resultados de predição de uma série de modelos diferentes para obter melhores resultados de predição.

Para o mesmo problema, diferentes modelos podem resolver melhor o problema, mas assim como um cego toca o elefante, cada modelo pode ter obtido parte da verdade dos dados, mas não toda a verdade. Ao reunir várias opiniões, é possível obter uma descrição mais precisa dos dados. Sob esse tipo de pensamento, pode-se dizer que integrar muitos modelos juntos irá inevitavelmente derrotar qualquer modelo único.

Para integração de modelos, a diversidade dos modelos usados ​​é muito importante. Diversidade é força.O modelo utilizado para a integração deve ser o melhor possível e, ao mesmo tempo, o mais diferente possível. Para a mesma rede, use inicializações aleatórias diferentes para vários treinamentos independentes e, em seguida, integre, de modo que o significado não seja grande. Uma abordagem melhor deve ser usar modelos com arquiteturas muito diferentes para integrar, de modo que os desvios de cada modelo estejam em direções diferentes, e a integração se compensará, e o resultado será mais estável e preciso.

Pegue o problema de classificação como exemplo. Primeiro, temos alguns modelos diferentes:

`` `python
preds_a = model_a.predict (x_val)
preds_b = model_b.predict (x_val)
preds_c = model_c.predict (x_val)
preds_d = model_d.predict (x_val)
`` `

Existem muitas maneiras diferentes de integrá-los. A maneira mais fácil é calcular a média:

`` `python
final_preds = 0,25 * (preds_a + preds_b + preds_c + preds_d)
`` `

Como haverá lacunas no desempenho de cada modelo, a melhor forma é a média ponderada:

`` `python
final_preds = 0,5 * preds_a + 0,25 * preds_b + 0,1 * preds_c + 0,15 * preds_d
`` `

O peso pode ser obtido com base na experiência, pesquisa aleatória ou outros algoritmos de otimização.